# Binary Classification with Bank Marketing Dataset

## Professional Machine Learning Pipeline with Advanced Feature Engineering and Hyperparameter Optimization

### Overview
This notebook presents a comprehensive machine learning solution for binary classification using a bank marketing dataset. The goal is to predict whether a client will subscribe to a term deposit based on various demographic, financial, and campaign-related features.

### Key Features:
- **Comprehensive Exploratory Data Analysis (EDA)**
- **Advanced Feature Engineering** with domain-specific transformations
- **Multi-Model Approach** using LightGBM, XGBoost, and CatBoost
- **Automated Hyperparameter Optimization** with Optuna
- **Model Ensembling** with optimized blending weights
- **Cross-Validation** for robust performance estimation

### Methodology:
1. Data Loading and Initial Analysis
2. Exploratory Data Analysis
3. Feature Engineering and Preprocessing
4. Model Development with Hyperparameter Tuning
5. Model Ensembling and Final Predictions

---


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
import io
import sys
import contextlib
from tqdm.auto import tqdm

# Machine Learning Libraries
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
from sklearn.preprocessing import RobustScaler
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier, Pool, CatBoostError

# Hyperparameter Optimization
import optuna
from optuna.samplers import TPESampler

# Configure display and warnings
warnings.filterwarnings("ignore")
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Set random seeds for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✅ All libraries imported successfully!")
print(f"📊 Using random state: {RANDOM_STATE}")


## 1. Data Loading and Configuration

Let's load the dataset and set up our configuration parameters.


In [ ]:
# Configuration Parameters
CONFIG = {
    'target_col': 'y',
    'id_col': 'id',
    'n_splits': 5,
    'random_state': RANDOM_STATE,
    'optuna_trials': 50,  # Number of trials for hyperparameter optimization
    'early_stopping_rounds': 100,
    'with_duration': True,  # Toggle duration-based features
    'te_smooth_m': 50.0,    # Target encoding smoothing parameter
    'verbose': True
}

# Context manager to suppress model training outputs when needed
@contextlib.contextmanager
def suppress_output():
    """Context manager to suppress stdout/stderr from libraries"""
    saved_stdout, saved_stderr = sys.stdout, sys.stderr
    try:
        sys.stdout, sys.stderr = io.StringIO(), io.StringIO()
        yield
    finally:
        sys.stdout, sys.stderr = saved_stdout, saved_stderr

# Silence Optuna's own logging for cleaner output
optuna.logging.set_verbosity(optuna.logging.WARNING)

print("⚙️ Configuration loaded successfully!")


In [ ]:
# Load datasets
print("📁 Loading datasets...")

try:
    # Load training data
    data_train = pd.read_csv("data/train.csv")
    print(f"   Training data loaded: {data_train.shape}")
    
    # Load test data  
    data_test = pd.read_csv("data/test.csv")
    print(f"   Test data loaded: {data_test.shape}")
    
    # Load additional bank data if available
    try:
        bank_full = pd.read_csv("data/bank-full.csv", sep=";")
        print(f"   Bank-full data loaded: {bank_full.shape}")
        
        # Convert target variable and combine datasets
        bank_full[CONFIG['target_col']] = bank_full[CONFIG['target_col']].map({'yes': 1, 'no': 0})
        
        # Remove ID column from training data if present
        if CONFIG['id_col'] in data_train.columns:
            data_train = data_train.drop(CONFIG['id_col'], axis=1)
            
        # Combine datasets for more training data
        data_train = pd.concat([bank_full, data_train], ignore_index=True)
        print(f"   Combined training data: {data_train.shape}")
        
    except FileNotFoundError:
        print("   bank-full.csv not found, proceeding with train.csv only")
        if CONFIG['id_col'] in data_train.columns:
            data_train = data_train.drop(CONFIG['id_col'], axis=1)
    
    print("✅ Data loading completed!")
    
    # Basic dataset info
    print(f"\n📊 Dataset Overview:")
    print(f"   Training samples: {len(data_train):,}")
    print(f"   Test samples: {len(data_test):,}")
    print(f"   Features: {data_train.shape[1] - 1}")  # -1 for target column
    
    # Check target distribution
    if CONFIG['target_col'] in data_train.columns:
        target_dist = data_train[CONFIG['target_col']].value_counts(normalize=True)
        print(f"   Target distribution: {target_dist.to_dict()}")
        
except FileNotFoundError as e:
    print(f"❌ Error loading data: {e}")
    raise


## 2. Exploratory Data Analysis (EDA)

Let's perform a comprehensive analysis of our dataset to understand the distribution of features, identify outliers, and discover patterns that will inform our feature engineering strategy.


In [ ]:
# Basic data exploration
print("🔍 Basic Data Exploration")
print("=" * 50)

# Display basic info
print(f"Dataset shape: {data_train.shape}")
print(f"Memory usage: {data_train.memory_usage().sum() / 1024**2:.2f} MB")

# Data types and missing values
print("\n📋 Data Types and Missing Values:")
print("-" * 40)
info_df = pd.DataFrame({
    'Data Type': data_train.dtypes,
    'Missing Values': data_train.isnull().sum(),
    'Missing %': (data_train.isnull().sum() / len(data_train)) * 100,
    'Unique Values': data_train.nunique()
})
print(info_df)

# Check for duplicates
duplicates = data_train.duplicated().sum()
print(f"\n🔄 Duplicate rows: {duplicates} ({duplicates/len(data_train)*100:.2f}%)")

# Target variable analysis
print(f"\n🎯 Target Variable ('{CONFIG['target_col']}') Analysis:")
print("-" * 40)
target_counts = data_train[CONFIG['target_col']].value_counts()
target_pct = data_train[CONFIG['target_col']].value_counts(normalize=True) * 100

for val, count in target_counts.items():
    print(f"   Class {val}: {count:,} samples ({target_pct[val]:.1f}%)")
    
class_ratio = target_counts[0] / target_counts[1] if len(target_counts) > 1 else 1
print(f"   Class imbalance ratio: {class_ratio:.2f}:1")


In [ ]:
# Identify numeric and categorical features
numeric_features = data_train.select_dtypes(include=[np.number]).columns.tolist()
if CONFIG['target_col'] in numeric_features:
    numeric_features.remove(CONFIG['target_col'])

categorical_features = data_train.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"📊 Feature Types:")
print(f"   Numeric features ({len(numeric_features)}): {numeric_features}")
print(f"   Categorical features ({len(categorical_features)}): {categorical_features}")

# Detailed analysis of numeric features
print("\n📈 Numeric Features Analysis:")
print("-" * 50)
numeric_stats = data_train[numeric_features].describe()
print(numeric_stats)

# Outlier detection for numeric features
print("\n🎯 Outlier Detection (IQR Method):")
print("-" * 40)
outlier_summary = {}

for col in numeric_features:
    Q1 = data_train[col].quantile(0.25)
    Q3 = data_train[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = ((data_train[col] < lower_bound) | (data_train[col] > upper_bound)).sum()
    outlier_pct = (outliers / len(data_train)) * 100
    
    outlier_summary[col] = {
        'count': outliers,
        'percentage': outlier_pct,
        'bounds': (lower_bound, upper_bound)
    }
    
    print(f"   {col:>12}: {outliers:,} outliers ({outlier_pct:.1f}%)")

# Skewness analysis
print("\n📐 Skewness Analysis:")
print("-" * 40)
for col in numeric_features:
    skewness = stats.skew(data_train[col])
    print(f"   {col:>12}: {skewness:.3f} {'(highly skewed)' if abs(skewness) > 1 else '(moderately skewed)' if abs(skewness) > 0.5 else '(normal)'}")


In [ ]:
# Categorical features analysis
print("🏷️ Categorical Features Analysis:")
print("-" * 50)

cat_analysis = {}
for col in categorical_features:
    unique_vals = data_train[col].nunique()
    mode_val = data_train[col].mode().iloc[0] if len(data_train[col].mode()) > 0 else 'N/A'
    mode_pct = (data_train[col] == mode_val).mean() * 100
    
    cat_analysis[col] = {
        'unique_values': unique_vals,
        'mode': mode_val,
        'mode_percentage': mode_pct
    }
    
    print(f"   {col:>12}: {unique_vals} unique values, mode='{mode_val}' ({mode_pct:.1f}%)")
    
    # Show value counts for features with reasonable number of categories
    if unique_vals <= 10:
        print(f"                 Distribution:")
        value_counts = data_train[col].value_counts()
        for val, count in value_counts.head().items():
            pct = (count / len(data_train)) * 100
            print(f"                   '{val}': {count:,} ({pct:.1f}%)")
        if len(value_counts) > 5:
            print(f"                   ... and {len(value_counts) - 5} more categories")
        print()


In [ ]:
# Visualizations for EDA
def create_eda_plots():
    """Create comprehensive EDA visualizations"""
    
    # Set up the plotting style
    plt.rcParams['figure.figsize'] = (15, 10)
    
    # 1. Target distribution
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Target Variable Analysis', fontsize=16, fontweight='bold')
    
    # Target counts
    target_counts = data_train[CONFIG['target_col']].value_counts()
    axes[0,0].bar(target_counts.index, target_counts.values, color=['lightcoral', 'lightblue'])
    axes[0,0].set_title('Target Distribution (Counts)')
    axes[0,0].set_xlabel('Target Class')
    axes[0,0].set_ylabel('Count')
    for i, v in enumerate(target_counts.values):
        axes[0,0].text(i, v + max(target_counts.values)*0.01, str(v), ha='center', va='bottom')
    
    # Target percentages (pie chart)
    axes[0,1].pie(target_counts.values, labels=target_counts.index, autopct='%1.1f%%', 
                  colors=['lightcoral', 'lightblue'], startangle=90)
    axes[0,1].set_title('Target Distribution (Percentages)')
    
    # Age distribution by target
    if 'age' in data_train.columns:
        for target_val in data_train[CONFIG['target_col']].unique():
            data_subset = data_train[data_train[CONFIG['target_col']] == target_val]
            axes[1,0].hist(data_subset['age'], alpha=0.6, label=f'Target={target_val}', bins=30)
        axes[1,0].set_title('Age Distribution by Target')
        axes[1,0].set_xlabel('Age')
        axes[1,0].set_ylabel('Frequency')
        axes[1,0].legend()
    
    # Duration vs target (if available)
    if 'duration' in data_train.columns:
        data_train.boxplot(column='duration', by=CONFIG['target_col'], ax=axes[1,1])
        axes[1,1].set_title('Duration Distribution by Target')
        axes[1,1].set_xlabel('Target Class')
        axes[1,1].set_ylabel('Duration')
    
    plt.tight_layout()
    plt.show()
    
    # 2. Correlation matrix for numeric features
    if len(numeric_features) > 1:
        plt.figure(figsize=(12, 10))
        correlation_matrix = data_train[numeric_features + [CONFIG['target_col']]].corr()
        mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
        sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
                   square=True, fmt='.2f', cbar_kws={"shrink": .8})
        plt.title('Feature Correlation Matrix', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.show()
    
    # 3. Distribution plots for key numeric features
    key_numeric = numeric_features[:6]  # Show first 6 numeric features
    if key_numeric:
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        axes = axes.ravel()
        
        for i, col in enumerate(key_numeric):
            if i < len(axes):
                # Histogram with KDE
                axes[i].hist(data_train[col], bins=30, alpha=0.7, color='skyblue', density=True)
                
                # Add KDE line
                try:
                    from scipy.stats import gaussian_kde
                    kde = gaussian_kde(data_train[col].dropna())
                    x_range = np.linspace(data_train[col].min(), data_train[col].max(), 100)
                    axes[i].plot(x_range, kde(x_range), 'r-', linewidth=2)
                except:
                    pass
                
                axes[i].set_title(f'{col} Distribution')
                axes[i].set_xlabel(col)
                axes[i].set_ylabel('Density')
        
        # Remove empty subplots
        for i in range(len(key_numeric), len(axes)):
            fig.delaxes(axes[i])
        
        plt.suptitle('Numeric Features Distribution', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.show()

# Create EDA plots
create_eda_plots()


## 3. Advanced Feature Engineering

Based on our EDA findings, we'll implement sophisticated feature engineering techniques including:
- **Domain-specific transformations** for banking/marketing features
- **Target encoding** with proper cross-validation to prevent leakage
- **Frequency encoding** for categorical variables
- **Interaction features** between related variables
- **Cyclical encoding** for temporal features
- **Outlier handling and normalization**


In [ ]:
# Feature Engineering Utility Functions
def month_to_num(s):
    """Convert month names to numbers"""
    m = str(s).strip().lower()
    order = dict(jan=1, feb=2, mar=3, apr=4, may=5, jun=6,
                jul=7, aug=8, sep=9, oct=10, nov=11, dec=12)
    return order.get(m, np.nan)

def add_domain_features(df: pd.DataFrame, with_duration: bool = True) -> pd.DataFrame:
    """
    Add domain-specific features for banking/marketing dataset
    
    Parameters:
    -----------
    df : DataFrame
        Input dataframe
    with_duration : bool
        Whether to include duration-based features
        
    Returns:
    --------
    DataFrame with new features added
    """
    out = df.copy()
    
    print("🔧 Adding domain-specific features...")
    
    # === Contact History Features ===
    if 'pdays' in out.columns:
        print("   📞 Contact history features...")
        pdays = out["pdays"].copy()
        
        # Binary flag for previous contact
        out["contacted_before"] = (pdays != 999).astype(int)
        out["contacted_before"] = (pdays != -1).astype(int) if (pdays == -1).any() else out["contacted_before"]
        
        # Days since last contact (handle 999/-1 as special values)
        pdays_masked = pdays.replace([999, -1], np.nan)
        out["days_since_last_contact"] = pdays_masked.fillna(999)
        out["pdays_log1p"] = np.log1p(pdays_masked).fillna(0)
        
        # Contact intensity (campaigns per day since last contact)
        if 'campaign' in out.columns:
            out["contact_intensity"] = (out["campaign"] / (pdays_masked + 1)).fillna(0)
    
    # === Previous Campaign Outcome Features ===
    if 'poutcome' in out.columns:
        print("   📊 Previous campaign outcome features...")
        pout = out["poutcome"].astype(str).str.lower()
        out["prev_success"] = (pout == "success").astype(int)
        out["prev_failure"] = (pout == "failure").astype(int)
        out["prev_unknown"] = (pout == "unknown").astype(int)
    
    # === Temporal Features ===
    if 'month' in out.columns:
        print("   📅 Temporal features...")
        mnum = out["month"].map(month_to_num)
        out["month_num"] = mnum
        
        # Cyclical encoding for month
        out["month_sin"] = np.sin(2 * np.pi * mnum / 12.0)
        out["month_cos"] = np.cos(2 * np.pi * mnum / 12.0)
        
        # Seasonal features
        out["is_summer"] = mnum.isin([6, 7, 8]).astype(int)
        out["is_q4"] = mnum.isin([10, 11, 12]).astype(int)
        out["is_spring"] = mnum.isin([3, 4, 5]).astype(int)
    
    # Day cyclical encoding (if exists)
    if 'day' in out.columns:
        day = out["day"].clip(1, 31)
        out["day_sin"] = np.sin(2 * np.pi * day / 31.0)
        out["day_cos"] = np.cos(2 * np.pi * day / 31.0)
    
    # === Campaign Features ===
    if 'campaign' in out.columns:
        print("   📢 Campaign features...")
        # Binned campaign attempts
        out["campaign_bins"] = pd.cut(out["campaign"],
                                      bins=[-np.inf, 1, 3, 6, np.inf],
                                      labels=["1", "2-3", "4-6", "7+"],
                                      ordered=True)
        
        # Campaign intensity categories
        out["campaign_high"] = (out["campaign"] >= out["campaign"].quantile(0.75)).astype(int)
        out["campaign_low"] = (out["campaign"] <= out["campaign"].quantile(0.25)).astype(int)
    
    # === Duration Features ===
    if with_duration and 'duration' in out.columns:
        print("   ⏱️ Duration features...")
        duration = out["duration"].clip(lower=0)
        out["duration_log1p"] = np.log1p(duration)
        out["duration_sqrt"] = np.sqrt(duration)
        
        # Duration categories
        out["duration_very_short"] = (duration <= 60).astype(int)  # <= 1 minute
        out["duration_short"] = ((duration > 60) & (duration <= 300)).astype(int)  # 1-5 minutes
        out["duration_medium"] = ((duration > 300) & (duration <= 600)).astype(int)  # 5-10 minutes
        out["duration_long"] = (duration > 600).astype(int)  # > 10 minutes
    
    # === Financial Features ===
    if 'balance' in out.columns:
        print("   💰 Financial features...")
        # Handle negative balances
        balance = out["balance"]
        shift = max(0, 1 - balance.min())  # Ensure positive values for log
        out["balance_log1p"] = np.log1p(balance + shift)
        out["has_positive_balance"] = (balance > 0).astype(int)
        out["has_negative_balance"] = (balance < 0).astype(int)
        
        # Balance categories
        out["balance_high"] = (balance >= balance.quantile(0.8)).astype(int)
        out["balance_low"] = (balance <= balance.quantile(0.2)).astype(int)
    
    # === Interaction Features ===
    print("   🔗 Interaction features...")
    
    # Housing and loan combination
    if {'housing', 'loan'}.issubset(out.columns):
        out["housing_loan_combo"] = (out["housing"].astype(str).str.lower() + "_" +
                                     out["loan"].astype(str).str.lower())
    
    # Job and education interaction
    if {'job', 'education'}.issubset(out.columns):
        out["job_x_education"] = (out["job"].astype(str).str.lower() + "__" +
                                  out["education"].astype(str).str.lower())
    
    # Contact history and campaign interaction
    if 'contacted_before' in out.columns and 'campaign' in out.columns:
        out["recency_x_campaign"] = out["contacted_before"] * out["campaign"]
    
    # Previous success and current campaign
    if 'prev_success' in out.columns and 'campaign' in out.columns:
        out["prev_success_x_campaign"] = out["prev_success"] * out["campaign"]
    
    # Duration and contact interaction
    if with_duration and 'contacted_before' in out.columns and 'duration_log1p' in out.columns:
        out["recent_and_long"] = out["contacted_before"] * out["duration_log1p"]
    
    # Age groups
    if 'age' in out.columns:
        print("   👥 Age group features...")
        age = out["age"]
        out["age_young"] = (age < 30).astype(int)
        out["age_middle"] = ((age >= 30) & (age < 50)).astype(int)
        out["age_senior"] = ((age >= 50) & (age < 65)).astype(int)
        out["age_elderly"] = (age >= 65).astype(int)
    
    print(f"   ✅ Added {len(out.columns) - len(df.columns)} new features")
    return out

# Apply domain feature engineering
print("🚀 Starting Feature Engineering Pipeline...")
train_engineered = add_domain_features(data_train, CONFIG['with_duration'])
test_engineered = add_domain_features(data_test, CONFIG['with_duration'])


In [ ]:
# Advanced Encoding Functions
def _make_te_map(X_tr_col: pd.Series, y_tr: pd.Series, m: float, prior: float):
    """Create target encoding mapping with smoothing"""
    stats = X_tr_col.to_frame("cat").assign(y=y_tr.values).groupby("cat")["y"].agg(["sum", "count"])
    te = (stats["sum"] + prior * m) / (stats["count"] + m)
    return te.to_dict()

def _apply_map(series: pd.Series, mapping: dict, default_val: float):
    """Apply mapping to series with default value for unseen categories"""
    return series.map(mapping).fillna(default_val).astype(float)

def add_fold_encodings(X_tr, y_tr, X_va, X_te, te_cols, fe_cols, te_smooth_m=50.0, strict_freq=True):
    """
    Add fold-safe target and frequency encodings
    
    Parameters:
    -----------
    X_tr, X_va, X_te : DataFrame
        Training, validation, and test sets
    y_tr : Series
        Training targets
    te_cols : list
        Columns for target encoding
    fe_cols : list  
        Columns for frequency encoding
    te_smooth_m : float
        Smoothing parameter for target encoding
    strict_freq : bool
        Whether to use strict frequency encoding (0 for unseen) or smoothed
        
    Returns:
    --------
    Tuple of encoded DataFrames (X_tr_e, X_va_e, X_te_e)
    """
    X_tr_e = X_tr.copy()
    X_va_e = X_va.copy()
    X_te_e = X_te.copy()
    
    # Target Encoding (with smoothing to prevent overfitting)
    prior = y_tr.mean()
    for col in te_cols:
        if col not in X_tr_e.columns:
            continue
        
        te_map = _make_te_map(X_tr_e[col].astype(str), y_tr, te_smooth_m, prior)
        default_val = prior
        
        X_tr_e[f"{col}_te"] = _apply_map(X_tr_e[col].astype(str), te_map, default_val)
        X_va_e[f"{col}_te"] = _apply_map(X_va_e[col].astype(str), te_map, default_val)
        X_te_e[f"{col}_te"] = _apply_map(X_te_e[col].astype(str), te_map, default_val)
    
    # Frequency Encoding
    for col in fe_cols:
        if col not in X_tr_e.columns:
            continue
        
        tr_counts = X_tr_e[col].astype(str).value_counts(dropna=False)
        tr_freq = (tr_counts / tr_counts.sum()).to_dict()
        default_freq = 0.0 if strict_freq else (1.0 / max(1, len(tr_counts)))
        
        X_tr_e[f"{col}_freq"] = X_tr_e[col].astype(str).map(tr_freq).fillna(default_freq).astype(float)
        X_va_e[f"{col}_freq"] = X_va_e[col].astype(str).map(tr_freq).fillna(default_freq).astype(float)
        X_te_e[f"{col}_freq"] = X_te_e[col].astype(str).map(tr_freq).fillna(default_freq).astype(float)
    
    return X_tr_e, X_va_e, X_te_e

print("🎯 Advanced encoding functions ready!")
print("   Target encoding: Prevents overfitting with smoothing")
print("   Frequency encoding: Captures category importance")


In [ ]:
# Data Preprocessing and Setup
print("🔄 Preprocessing and Data Setup...")

# Handle target variable encoding
train = train_engineered.copy()
test = test_engineered.copy()

# Ensure target is properly encoded
if train[CONFIG['target_col']].dtype == "O":
    train[CONFIG['target_col']] = train[CONFIG['target_col']].str.strip().str.lower().map({"yes": 1, "no": 0})

# Handle ID column for test set
if CONFIG['id_col'] not in test.columns:
    test[CONFIG['id_col']] = np.arange(len(test))

# Identify categorical columns and cast to category dtype for efficient handling
all_obj_cols = [c for c in train.columns if train[c].dtype == "O" and c != CONFIG['target_col']]
print(f"   Converting {len(all_obj_cols)} object columns to category dtype...")

for col in all_obj_cols:
    train[col] = train[col].astype("category")
    if col in test.columns:
        test[col] = test[col].astype("category")

# Handle existing category columns and align categories across train/test
all_cat_cols = [c for c in train.columns if str(train[c].dtype).startswith("category")]
print(f"   Aligning {len(all_cat_cols)} categorical columns across train/test...")

for col in all_cat_cols:
    if col in test.columns:
        # Get union of all categories
        train_cats = set(train[col].cat.categories.tolist()) if hasattr(train[col], 'cat') else set(train[col].unique())
        test_cats = set(test[col].cat.categories.tolist()) if hasattr(test[col], 'cat') else set(test[col].unique())
        all_cats = sorted(list(train_cats | test_cats))
        
        # Set categories
        train[col] = train[col].cat.set_categories(all_cats)
        test[col] = test[col].cat.set_categories(all_cats)

# Define feature sets
features = [c for c in train.columns if c not in {CONFIG['target_col'], CONFIG['id_col']}]
categorical_features = [c for c in features if str(train[c].dtype).startswith("category")]
numeric_features = [c for c in features if c not in categorical_features]

print(f"   📊 Feature Summary:")
print(f"      Total features: {len(features)}")
print(f"      Categorical: {len(categorical_features)}")
print(f"      Numeric: {len(numeric_features)}")

# Prepare data matrices
X_full = train[features].copy()
y_full = train[CONFIG['target_col']].astype(int)
X_test_full = test[features].copy()

# Features for different models
# LightGBM and CatBoost can handle categorical features natively
categorical_indices = [X_full.columns.get_loc(c) for c in categorical_features]

# XGBoost needs only numeric features (we'll handle categoricals with encodings)
xgb_features = [c for c in features if not str(X_full[c].dtype).startswith("category")]

print(f"   🎯 Model-specific features:")
print(f"      LightGBM/CatBoost: {len(features)} features ({len(categorical_features)} categorical)")
print(f"      XGBoost: {len(xgb_features)} features (numeric only)")

print("✅ Data preprocessing completed!")


## 4. Model Development with Hyperparameter Optimization

We'll use Optuna to automatically optimize hyperparameters for three powerful gradient boosting algorithms:
- **LightGBM**: Fast and efficient with native categorical support
- **XGBoost**: Robust and well-tested with excellent performance
- **CatBoost**: Advanced categorical handling and built-in regularization

Each model will be optimized using cross-validation to prevent overfitting and ensure robust performance.


In [ ]:
# Cross-Validation Setup
skf = StratifiedKFold(n_splits=CONFIG['n_splits'], shuffle=True, random_state=CONFIG['random_state'])

print(f"🔀 Cross-Validation Setup:")
print(f"   Strategy: Stratified {CONFIG['n_splits']}-Fold")
print(f"   Random State: {CONFIG['random_state']}")

# Define columns for advanced encodings
te_cols = ['job', 'education', 'contact', 'month', 'poutcome', 'marital']  # Target encoding
fe_cols = ['job', 'education', 'contact', 'month', 'poutcome', 'marital']  # Frequency encoding

# Add interaction features to encoding lists
interaction_cols = ['housing_loan_combo', 'job_x_education', 'campaign_bins']
te_cols.extend([col for col in interaction_cols if col in train.columns])
fe_cols.extend([col for col in interaction_cols if col in train.columns])

print(f"   Target Encoding Columns: {len(te_cols)} features")
print(f"   Frequency Encoding Columns: {len(fe_cols)} features")

# Model-specific objective functions for Optuna
def tune_lightgbm(trial):
    """Optuna objective function for LightGBM hyperparameter tuning"""
    params = {
        "objective": "binary",
        "metric": "auc", 
        "boosting_type": "gbdt",
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 31, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 10),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 200),
        "lambda_l1": trial.suggest_float("lambda_l1", 0.0, 5.0),
        "lambda_l2": trial.suggest_float("lambda_l2", 0.0, 5.0),
        "verbose": -1,
        "is_unbalance": True,
        "seed": CONFIG['random_state'],
        "device_type": "cpu"  # Ensure CPU usage for stability
    }
    
    oof_preds = np.zeros(len(X_full))
    
    for fold, (tr_idx, va_idx) in enumerate(skf.split(X_full, y_full)):
        X_tr, y_tr = X_full.iloc[tr_idx].copy(), y_full.iloc[tr_idx].copy()
        X_va, y_va = X_full.iloc[va_idx].copy(), y_full.iloc[va_idx].copy()
        X_te = X_test_full.copy()
        
        # Add fold-safe encodings
        X_tr_e, X_va_e, _ = add_fold_encodings(
            X_tr, y_tr, X_va, X_te, te_cols, fe_cols, CONFIG['te_smooth_m'], strict_freq=True
        )
        
        fold_features = list(X_tr_e.columns)
        fold_cat_features = [c for c in fold_features if (c in categorical_features and c in X_tr_e.columns)]
        
        # Create datasets
        dtrain = lgb.Dataset(X_tr_e[fold_features], label=y_tr, 
                           categorical_feature=fold_cat_features, free_raw_data=False)
        dvalid = lgb.Dataset(X_va_e[fold_features], label=y_va,
                           categorical_feature=fold_cat_features, reference=dtrain, free_raw_data=False)
        
        # Train model with suppressed output
        with suppress_output():
            model = lgb.train(
                params,
                dtrain,
                valid_sets=[dvalid],
                callbacks=[
                    lgb.early_stopping(CONFIG['early_stopping_rounds'], verbose=False),
                    lgb.log_evaluation(0)
                ]
            )
        
        oof_preds[va_idx] = model.predict(X_va_e[fold_features], num_iteration=model.best_iteration)
    
    return roc_auc_score(y_full, oof_preds)

def tune_xgboost(trial):
    """Optuna objective function for XGBoost hyperparameter tuning"""
    params = {
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "tree_method": "hist",
        "verbosity": 0,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "lambda": trial.suggest_float("lambda", 0.0, 5.0),
        "alpha": trial.suggest_float("alpha", 0.0, 5.0),
        "scale_pos_weight": 1.0,
        "device": "cpu"  # Ensure CPU usage for stability
    }
    
    oof_preds = np.zeros(len(X_full))
    
    for tr_idx, va_idx in skf.split(X_full, y_full):
        X_tr, y_tr = X_full.iloc[tr_idx].copy(), y_full.iloc[tr_idx].copy()
        X_va, y_va = X_full.iloc[va_idx].copy(), y_full.iloc[va_idx].copy()
        X_te = X_test_full.copy()
        
        # Add fold-safe encodings and use only numeric features for XGBoost
        X_tr_e, X_va_e, _ = add_fold_encodings(
            X_tr, y_tr, X_va, X_te, te_cols, fe_cols, CONFIG['te_smooth_m'], strict_freq=True
        )
        
        # Filter to numeric features only
        fold_xgb_features = [c for c in X_tr_e.columns if not str(X_tr_e[c].dtype).startswith("category")]
        
        dtrain = xgb.DMatrix(X_tr_e[fold_xgb_features], label=y_tr)
        dvalid = xgb.DMatrix(X_va_e[fold_xgb_features], label=y_va)
        
        # Train model with suppressed output
        with suppress_output():
            model = xgb.train(
                params,
                dtrain,
                num_boost_round=5000,
                evals=[(dvalid, "valid")],
                early_stopping_rounds=CONFIG['early_stopping_rounds'],
                verbose_eval=False
            )
        
        oof_preds[va_idx] = model.predict(dvalid, iteration_range=(0, model.best_iteration))
    
    return roc_auc_score(y_full, oof_preds)

def tune_catboost(trial):
    """Optuna objective function for CatBoost hyperparameter tuning"""
    params = {
        "iterations": 5000,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
        "depth": trial.suggest_int("depth", 4, 10),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1.0, 10.0),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "bootstrap_type": "Bernoulli",
        "eval_metric": "AUC",
        "task_type": "CPU",  # Use CPU for stability
        "random_seed": CONFIG['random_state'],
        "verbose": False,
        "logging_level": "Silent",
        "allow_writing_files": False,
        "use_best_model": True
    }
    
    oof_preds = np.zeros(len(X_full))
    
    for tr_idx, va_idx in skf.split(X_full, y_full):
        X_tr, y_tr = X_full.iloc[tr_idx].copy(), y_full.iloc[tr_idx].copy()
        X_va, y_va = X_full.iloc[va_idx].copy(), y_full.iloc[va_idx].copy()
        X_te = X_test_full.copy()
        
        # Add fold-safe encodings
        X_tr_e, X_va_e, _ = add_fold_encodings(
            X_tr, y_tr, X_va, X_te, te_cols, fe_cols, CONFIG['te_smooth_m'], strict_freq=True
        )
        
        fold_features = list(X_tr_e.columns)
        fold_cat_features = [c for c in fold_features if (c in categorical_features and c in X_tr_e.columns)]
        
        # Create pools
        train_pool = Pool(X_tr_e[fold_features], y_tr, cat_features=fold_cat_features)
        valid_pool = Pool(X_va_e[fold_features], y_va, cat_features=fold_cat_features)
        
        # Train model with suppressed output
        with suppress_output():
            model = CatBoostClassifier(**params)
            model.fit(train_pool, eval_set=valid_pool, 
                     early_stopping_rounds=CONFIG['early_stopping_rounds'], verbose=False)
        
        oof_preds[va_idx] = model.predict_proba(valid_pool)[:, 1]
    
    return roc_auc_score(y_full, oof_preds)

print("🎯 Hyperparameter optimization functions ready!")
print("   Each model uses cross-validation with fold-safe encodings")
print("   Target metric: ROC-AUC")


In [ ]:
# Hyperparameter Optimization with Optuna
print("🚀 Starting Hyperparameter Optimization...")
print("="*60)

# Set up Optuna sampler for better optimization
sampler = TPESampler(seed=CONFIG['random_state'])

# 1. LightGBM Optimization
print("\n📈 Optimizing LightGBM...")
study_lgb = optuna.create_study(direction="maximize", sampler=sampler)
study_lgb.optimize(tune_lightgbm, n_trials=CONFIG['optuna_trials'], show_progress_bar=True)

lgb_best_score = study_lgb.best_value
lgb_best_params = study_lgb.best_params
print(f"   ✅ LightGBM Best Score: {lgb_best_score:.5f}")
print(f"   🔧 Best Params: {lgb_best_params}")

# 2. XGBoost Optimization  
print("\n📊 Optimizing XGBoost...")
study_xgb = optuna.create_study(direction="maximize", sampler=sampler)
study_xgb.optimize(tune_xgboost, n_trials=CONFIG['optuna_trials'], show_progress_bar=True)

xgb_best_score = study_xgb.best_value
xgb_best_params = study_xgb.best_params
print(f"   ✅ XGBoost Best Score: {xgb_best_score:.5f}")
print(f"   🔧 Best Params: {xgb_best_params}")

# 3. CatBoost Optimization
print("\n🐱 Optimizing CatBoost...")
study_cat = optuna.create_study(direction="maximize", sampler=sampler) 
study_cat.optimize(tune_catboost, n_trials=CONFIG['optuna_trials'], show_progress_bar=True)

cat_best_score = study_cat.best_value
cat_best_params = study_cat.best_params
print(f"   ✅ CatBoost Best Score: {cat_best_score:.5f}")
print(f"   🔧 Best Params: {cat_best_params}")

# Summary of optimization results
print("\n" + "="*60)
print("🏆 OPTIMIZATION RESULTS SUMMARY")
print("="*60)
print(f"LightGBM Score: {lgb_best_score:.5f}")
print(f"XGBoost Score:  {xgb_best_score:.5f}")
print(f"CatBoost Score: {cat_best_score:.5f}")

best_single_model = max([
    ("LightGBM", lgb_best_score),
    ("XGBoost", xgb_best_score), 
    ("CatBoost", cat_best_score)
], key=lambda x: x[1])

print(f"\n🥇 Best Single Model: {best_single_model[0]} ({best_single_model[1]:.5f})")
print("✅ Hyperparameter optimization completed!")


## 5. Model Ensembling and Final Predictions

Now we'll train the final models with optimized hyperparameters and create an ensemble by:
1. **Training each model** with best parameters from optimization
2. **Generating out-of-fold predictions** for proper ensemble training
3. **Optimizing blend weights** using Optuna
4. **Creating final predictions** on the test set

This ensemble approach typically outperforms any single model by combining their strengths and reducing individual model biases.


In [ ]:
# Final Model Training Function
def train_final_models():
    """Train final models with optimized hyperparameters and generate OOF predictions"""
    
    print("🏗️ Training Final Models with Optimized Hyperparameters...")
    print("="*60)
    
    # Initialize storage for out-of-fold predictions and test predictions
    oof_predictions = {}
    test_predictions = {}
    model_scores = {}
    
    # Model configurations with optimized parameters
    model_configs = [
        ("LightGBM", lgb_best_params, "lightgbm"),
        ("XGBoost", xgb_best_params, "xgboost"), 
        ("CatBoost", cat_best_params, "catboost")
    ]
    
    total_steps = len(model_configs) * CONFIG['n_splits']
    
    with tqdm(total=total_steps, desc="Training ensemble models", leave=True) as pbar:
        for model_name, best_params, model_type in model_configs:
            print(f"\n🔧 Training {model_name}...")
            
            # Initialize prediction arrays
            oof_preds = np.zeros(len(X_full))
            test_preds = np.zeros(len(X_test_full))
            fold_scores = []
            
            # Cross-validation training
            for fold, (tr_idx, va_idx) in enumerate(skf.split(X_full, y_full)):
                X_tr, y_tr = X_full.iloc[tr_idx].copy(), y_full.iloc[tr_idx].copy()
                X_va, y_va = X_full.iloc[va_idx].copy(), y_full.iloc[va_idx].copy() 
                X_te = X_test_full.copy()
                
                # Add fold-safe encodings
                X_tr_e, X_va_e, X_te_e = add_fold_encodings(
                    X_tr, y_tr, X_va, X_te, te_cols, fe_cols, CONFIG['te_smooth_m'], strict_freq=True
                )
                
                # Train model based on type
                if model_type == "lightgbm":
                    fold_features = list(X_tr_e.columns)
                    fold_cat_features = [c for c in fold_features if (c in categorical_features and c in X_tr_e.columns)]
                    
                    dtrain = lgb.Dataset(X_tr_e[fold_features], label=y_tr,
                                       categorical_feature=fold_cat_features, free_raw_data=False)
                    dvalid = lgb.Dataset(X_va_e[fold_features], label=y_va,
                                       categorical_feature=fold_cat_features, reference=dtrain, free_raw_data=False)
                    
                    with suppress_output():
                        model = lgb.train(
                            {**best_params, "objective": "binary", "metric": "auc", "verbose": -1, "seed": CONFIG['random_state']},
                            dtrain,
                            valid_sets=[dvalid],
                            callbacks=[lgb.early_stopping(CONFIG['early_stopping_rounds'], verbose=False), lgb.log_evaluation(0)]
                        )
                    
                    oof_preds[va_idx] = model.predict(X_va_e[fold_features], num_iteration=model.best_iteration)
                    test_preds += model.predict(X_te_e[fold_features], num_iteration=model.best_iteration) / CONFIG['n_splits']
                
                elif model_type == "xgboost":
                    fold_xgb_features = [c for c in X_tr_e.columns if not str(X_tr_e[c].dtype).startswith("category")]
                    
                    dtrain = xgb.DMatrix(X_tr_e[fold_xgb_features], label=y_tr)
                    dvalid = xgb.DMatrix(X_va_e[fold_xgb_features], label=y_va)
                    
                    params_full = {**best_params, "objective": "binary:logistic", "eval_metric": "auc", "verbosity": 0}
                    
                    with suppress_output():
                        model = xgb.train(
                            params_full,
                            dtrain,
                            num_boost_round=5000,
                            evals=[(dvalid, "valid")],
                            early_stopping_rounds=CONFIG['early_stopping_rounds'],
                            verbose_eval=False
                        )
                    
                    oof_preds[va_idx] = model.predict(dvalid, iteration_range=(0, model.best_iteration))
                    test_preds += model.predict(xgb.DMatrix(X_te_e[fold_xgb_features]), 
                                               iteration_range=(0, model.best_iteration)) / CONFIG['n_splits']
                
                elif model_type == "catboost":
                    fold_features = list(X_tr_e.columns)
                    fold_cat_features = [c for c in fold_features if (c in categorical_features and c in X_tr_e.columns)]
                    
                    train_pool = Pool(X_tr_e[fold_features], y_tr, cat_features=fold_cat_features)
                    valid_pool = Pool(X_va_e[fold_features], y_va, cat_features=fold_cat_features)
                    
                    with suppress_output():
                        model = CatBoostClassifier(
                            **best_params, iterations=5000, eval_metric="AUC", verbose=False,
                            logging_level="Silent", allow_writing_files=False, use_best_model=True,
                            random_seed=CONFIG['random_state']
                        )
                        model.fit(train_pool, eval_set=valid_pool, 
                                 early_stopping_rounds=CONFIG['early_stopping_rounds'], verbose=False)
                    
                    oof_preds[va_idx] = model.predict_proba(valid_pool)[:, 1]
                    test_preds += model.predict_proba(Pool(X_te_e[fold_features], cat_features=fold_cat_features))[:, 1] / CONFIG['n_splits']
                
                # Calculate fold score
                fold_score = roc_auc_score(y_va, oof_preds[va_idx])
                fold_scores.append(fold_score)
                pbar.update(1)
            
            # Calculate final model score
            final_score = roc_auc_score(y_full, oof_preds)
            model_scores[model_name] = final_score
            
            # Store predictions
            oof_predictions[model_name] = oof_preds
            test_predictions[model_name] = test_preds
            
            print(f"   ✅ {model_name} CV Score: {final_score:.5f} (±{np.std(fold_scores):.5f})")
            print(f"      Fold Scores: {', '.join([f'{score:.5f}' for score in fold_scores])}")
    
    return oof_predictions, test_predictions, model_scores

# Train all final models
oof_preds, test_preds, model_scores = train_final_models()


In [ ]:
# Ensemble Weight Optimization
print("\n🎨 Optimizing Ensemble Weights...")
print("="*40)

# Convert predictions to numpy arrays for easier handling
oof_models_array = np.array([oof_preds[model] for model in ["LightGBM", "XGBoost", "CatBoost"]])
test_models_array = np.array([test_preds[model] for model in ["LightGBM", "XGBoost", "CatBoost"]])

def optimize_blend_weights(trial):
    """Optuna objective function for optimizing ensemble weights"""
    w_lgb = trial.suggest_float("w_lgb", 0.0, 1.0)
    w_xgb = trial.suggest_float("w_xgb", 0.0, 1.0)
    w_cat = trial.suggest_float("w_cat", 0.0, 1.0)
    
    # Normalize weights
    weights = np.array([w_lgb, w_xgb, w_cat])
    if weights.sum() == 0:
        weights = np.array([1.0, 1.0, 1.0])
    weights = weights / weights.sum()
    
    # Calculate blended predictions
    blended_oof = np.average(oof_models_array, axis=0, weights=weights)
    
    # Return ROC-AUC score
    return roc_auc_score(y_full, blended_oof)

# Optimize ensemble weights
study_blend = optuna.create_study(direction="maximize", sampler=TPESampler(seed=CONFIG['random_state']))
study_blend.optimize(optimize_blend_weights, n_trials=100, show_progress_bar=True)

# Get optimal weights
best_weights = np.array([
    study_blend.best_params["w_lgb"],
    study_blend.best_params["w_xgb"], 
    study_blend.best_params["w_cat"]
])

if best_weights.sum() == 0:
    best_weights = np.array([1.0, 1.0, 1.0])
best_weights = best_weights / best_weights.sum()

# Calculate final ensemble predictions
ensemble_oof = np.average(oof_models_array, axis=0, weights=best_weights)
ensemble_test = np.average(test_models_array, axis=0, weights=best_weights)
ensemble_score = roc_auc_score(y_full, ensemble_oof)

print(f"\n🎯 Ensemble Optimization Results:")
print(f"   Best Ensemble Score: {ensemble_score:.5f}")
print(f"   Optimal Weights:")
print(f"      LightGBM: {best_weights[0]:.3f}")
print(f"      XGBoost:  {best_weights[1]:.3f}")
print(f"      CatBoost: {best_weights[2]:.3f}")

# Compare with individual models
print(f"\n📊 Model Performance Comparison:")
print(f"   Individual Models:")
for model_name, score in model_scores.items():
    print(f"      {model_name}: {score:.5f}")
print(f"   Ensemble: {ensemble_score:.5f}")

improvement = ensemble_score - max(model_scores.values())
print(f"\n🚀 Ensemble Improvement: +{improvement:.5f} over best single model")

if improvement > 0:
    print("   ✅ Ensemble outperforms individual models!")
else:
    print("   ℹ️ Ensemble performs similarly to best individual model")


In [ ]:
# Generate Final Predictions and Submission
print("\n📄 Generating Final Submission...")
print("="*40)

# Create submission DataFrame
submission_df = pd.DataFrame({
    CONFIG['id_col']: test[CONFIG['id_col']].values,
    CONFIG['target_col']: ensemble_test
})

# Save probability predictions
prob_submission_path = "final_ensemble_predictions_probabilities.csv"
submission_df.to_csv(prob_submission_path, index=False)
print(f"✅ Probability predictions saved to: {prob_submission_path}")

# Create binary predictions (threshold = 0.5)
submission_binary = submission_df.copy()
submission_binary[CONFIG['target_col']] = (submission_binary[CONFIG['target_col']] >= 0.5).astype(int)

binary_submission_path = "final_ensemble_predictions_binary.csv"
submission_binary.to_csv(binary_submission_path, index=False)
print(f"✅ Binary predictions saved to: {binary_submission_path}")

# Show prediction statistics
print(f"\n📈 Prediction Statistics:")
print(f"   Probability predictions range: [{ensemble_test.min():.4f}, {ensemble_test.max():.4f}]")
print(f"   Mean prediction probability: {ensemble_test.mean():.4f}")
print(f"   Predicted positive class ratio: {(ensemble_test >= 0.5).mean():.3f}")

# Show first few predictions
print(f"\n👀 Sample Predictions:")
print(submission_df.head(10))

print(f"\n🎉 FINAL RESULTS SUMMARY")
print("="*50)
print(f"🔬 Dataset: Bank Marketing Binary Classification")
print(f"📊 Training Samples: {len(X_full):,}")
print(f"🧪 Test Samples: {len(X_test_full):,}")
print(f"🔧 Features: {len(features)}")
print(f"🎯 Best Single Model: {best_single_model[0]} ({best_single_model[1]:.5f})")
print(f"🏆 Final Ensemble Score: {ensemble_score:.5f}")
print(f"📈 Improvement: +{improvement:.5f}")
print(f"💾 Predictions saved to: {binary_submission_path}")
print("="*50)
print("✅ Analysis Complete! Ready for submission.")


## 6. Model Analysis and Feature Importance

Let's analyze our final ensemble to understand which features contributed most to the predictions and gain insights into the model's decision-making process.


In [ ]:
# Feature Importance Analysis
print("🔍 Analyzing Feature Importance...")

# Train a final LightGBM model on full dataset for feature importance analysis
# (LightGBM provides the most interpretable feature importance)
print("   Training analysis model on full dataset...")

# Prepare full dataset with encodings (using simple mean encoding for this analysis)
X_analysis = X_full.copy()
y_analysis = y_full.copy()

# Add simple target encoding for categorical features (for analysis only)
for col in categorical_features:
    if col in X_analysis.columns:
        target_mean = y_analysis.groupby(X_analysis[col]).mean()
        X_analysis[f"{col}_te_simple"] = X_analysis[col].map(target_mean).fillna(y_analysis.mean())

# Get numeric features for analysis
analysis_features = [c for c in X_analysis.columns if not str(X_analysis[c].dtype).startswith("category")]

# Train analysis model
analysis_params = {
    **lgb_best_params,
    "objective": "binary",
    "metric": "auc",
    "verbose": -1,
    "seed": CONFIG['random_state']
}

dtrain_analysis = lgb.Dataset(X_analysis[analysis_features], label=y_analysis, free_raw_data=False)

with suppress_output():
    analysis_model = lgb.train(
        analysis_params,
        dtrain_analysis,
        num_boost_round=500  # Reduced for analysis
    )

# Get feature importance
feature_importance = analysis_model.feature_importance(importance_type='gain')
feature_names = analysis_features

# Create feature importance dataframe
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importance
}).sort_values('importance', ascending=False).reset_index(drop=True)

# Display top 20 most important features
print(f"\n🏆 Top 20 Most Important Features:")
print("-" * 50)
for i, (_, row) in enumerate(importance_df.head(20).iterrows(), 1):
    print(f"   {i:2d}. {row['feature']:<25} {row['importance']:>8.0f}")

# Create feature importance visualization
plt.figure(figsize=(12, 8))
top_features = importance_df.head(15)
sns.barplot(data=top_features, y='feature', x='importance', palette='viridis')
plt.title('Top 15 Feature Importance (LightGBM)', fontsize=16, fontweight='bold')
plt.xlabel('Feature Importance (Gain)', fontsize=12)
plt.ylabel('Features', fontsize=12)
plt.tight_layout()
plt.show()

print("✅ Feature importance analysis completed!")


## 7. Conclusions and Methodology Summary

### 🎯 **Project Overview**
This notebook presented a comprehensive machine learning pipeline for binary classification of bank marketing data, achieving robust performance through advanced feature engineering and ensemble modeling.

### 🔧 **Key Methodologies Applied**

#### **1. Advanced Feature Engineering**
- **Domain-specific transformations**: Banking/marketing specific feature creation
- **Cyclical encoding**: Temporal features (month, day) using sine/cosine transformations  
- **Target encoding**: Categorical variables with cross-validation to prevent leakage
- **Interaction features**: Meaningful combinations of related variables
- **Financial indicators**: Balance categories, contact history analysis

#### **2. Robust Model Development**
- **Multi-model approach**: LightGBM, XGBoost, and CatBoost for diversity
- **Hyperparameter optimization**: Automated tuning with Optuna (50+ trials per model)
- **Cross-validation**: Stratified 5-fold CV for unbiased performance estimation
- **Early stopping**: Preventing overfitting during training

#### **3. Ensemble Learning**
- **Out-of-fold predictions**: Proper ensemble training without data leakage
- **Optimized blending**: Automated weight optimization using Optuna
- **Model diversity**: Leveraging strengths of different algorithms

### 📈 **Technical Achievements**
- **Professional code structure**: Modular, documented, and maintainable
- **Comprehensive EDA**: Deep understanding of data patterns and relationships
- **Advanced preprocessing**: Proper handling of categorical, numerical, and temporal features
- **Automated optimization**: Minimal manual hyperparameter tuning
- **Robust validation**: Cross-validated performance estimates

### 🚀 **Business Impact**
The resulting model can effectively predict customer subscription likelihood, enabling:
- **Targeted marketing campaigns**: Focus resources on high-probability prospects
- **Cost optimization**: Reduce wasted marketing spend on unlikely converts
- **Strategy insights**: Understand key factors influencing customer decisions

### 💡 **Key Insights**
From our feature importance analysis, the most predictive factors for customer subscription include:
- **Contact duration and history**: Previous interactions strongly predict success
- **Campaign timing**: Seasonal and monthly patterns matter significantly  
- **Customer demographics**: Age, job, and education level are key indicators
- **Financial status**: Balance and existing loan status influence decisions

This professional-grade solution demonstrates best practices in machine learning pipeline development and can serve as a template for similar binary classification problems in business contexts.
